## Maximize Sales

In [1]:
! pip install pulp


   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 682.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/17.7 MB 1.3 MB/s eta 0:00:14
   - -------------------------------------- 0.6/17.7 MB 5.8 MB/s eta 0:00:03
   ----- ---------------------------------- 2.6/17.7 MB 16.7 MB/s eta 0:00:01
   ----------- ---------------------------- 4.9/17.7 MB 22.3 MB/s eta 0:00:01
   ---------------- ----------------------- 7.1/17.7 MB 26.9 MB/s eta 0:00:01
   -------------------- ------------------- 9.2/17.7 MB 29.6 MB/s eta 0:00:01
   ------------------------ --------------- 11.0/17.7 MB 43.7 MB/s eta 0:00:01
   ----------------------------- ---------- 12.9/17.7 MB 46.9 MB/s eta 0:00:01
   ---------------------------------- ----- 15.1/17.7 MB 43.5 MB/s eta 0:00:01
   ---------------------------------------  17.4/17.7 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------- 17.7/17.7 MB 40.9 MB/s et

In [11]:
import pandas as pd
import pulp

def maximize_sales(input_csv= "Acme_Synthetic_Dataset.csv",output_csv= "Optimized_Sales_Results.csv"):


    # 1. Load the dataset
    df = pd.read_csv(input_csv)

    # 2. Initialize the LP problem to maximize sales
    prob = pulp.LpProblem('Maximize_Sales', pulp.LpMaximize)

    # 3. Create an LP variable for each segment's sales (no negative sales)
    sales_vars = pulp.LpVariable.dicts('Sales', df.index, lowBound=0)

    # 4. Set the objective function: sum of all sales variables
    prob += pulp.lpSum([sales_vars[i] for i in df.index]), "Total_Sales"

    # 5. Add trend constraints for each segment
    for idx, row in df.iterrows():
        min_sales = row['Initial_Sales'] * (1 + row['Min_Trend'] / 100.0)
        max_sales = row['Initial_Sales'] * (1 + row['Max_Trend'] / 100.0)
        prob += sales_vars[idx] >= min_sales, f"Min_Trend_{idx}"
        prob += sales_vars[idx] <= max_sales, f"Max_Trend_{idx}"

    # 6. Group data by Portfolio, Geography, Category, Brand to apply contribution constraints
    grouped = df.groupby(['Portfolio', 'Geography', 'Category', 'Brand'])

    # 7. Add contribution constraints for each segment within its brand group
    for name, group in grouped:
        indices = group.index
        total_sales = pulp.lpSum([sales_vars[i] for i in indices])
        for idx in indices:
            min_contrib = df.loc[idx, 'Min_Contribution'] / 100.0
            max_contrib = df.loc[idx, 'Max_Contribution'] / 100.0
            prob += sales_vars[idx] >= min_contrib * total_sales, f"Min_Contribution_{idx}"
            prob += sales_vars[idx] <= max_contrib * total_sales, f"Max_Contribution_{idx}"

    # 8. Solve the LP problem
    prob.solve()

    # 9. User-provided expected total sales
    expected_total_sales = float(input("Enter the expected total sales value: "))

    # 10. Construct results and compare
    results = []
    for idx, row in df.iterrows():
        new_sales = sales_vars[idx].varValue

        # Calculate the applied trend (%)
        initial_sales = row['Initial_Sales']
        if initial_sales != 0:
            applied_trend = (new_sales / initial_sales - 1) * 100.0
        else:
            applied_trend = 0.0

        # Calculate contribution (%) within the brand group
        brand_group = grouped.get_group((
            row['Portfolio'],
            row['Geography'],
            row['Category'],
            row['Brand']
        ))
        total_brand_sales = sum(sales_vars[i].varValue for i in brand_group.index)
        if total_brand_sales != 0:
            contribution = (new_sales / total_brand_sales) * 100.0
        else:
            contribution = 0.0

        # Store results in a list of dicts
        results.append({
            'Portfolio': row['Portfolio'],
            'Geography': row['Geography'],
            'Category': row['Category'],
            'Brand': row['Brand'],
            'Segment': row['Segment'],
            'Initial_Sales': initial_sales,
            'New_Sales': new_sales,
            'Margin (%)': row['Margin'],
            'Applied_Trend (%)': applied_trend,
            'Contribution (%)': contribution,
            'Min_Trend': row['Min_Trend'],
            'Max_Trend': row['Max_Trend'],
            'Min_Contribution': row['Min_Contribution'],
            'Max_Contribution': row['Max_Contribution']
        })

    # 11. Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # 12. Calculate total maximum sales
    total_max_sales = results_df['New_Sales'].sum()
    print(f"\nTotal Maximum Sales Achievable: ${total_max_sales:,.2f}")

    # 13. Compare with the expected sales
    if total_max_sales >= expected_total_sales:
        diff = total_max_sales - expected_total_sales
        print(f"The achievable sales meet/exceed the expected sales by ${diff:,.2f}.")
    else:
        diff = expected_total_sales - total_max_sales
        print(f"The achievable sales are below the expected by ${diff:,.2f}.")

    # 14. Display partial results
    print("\nOptimized Results (first 5 rows):")
    print(results_df.head(5))

    # 15. Export the results to a new CSV file
    results_df.to_csv(output_csv, index=False)
    print(f"\nResults have been saved to '{output_csv}'.")

maximize_sales(input_csv="Acme_Synthetic_Dataset.csv",output_csv="Optimized_Sales_Results.csv")



Total Maximum Sales Achievable: $2,853,990,961.86
The achievable sales meet/exceed the expected sales by $1,853,990,961.86.

Optimized Results (first 5 rows):
   Portfolio      Geography         Category           Brand    Segment  Initial_Sales  New_Sales  Margin (%)  Applied_Trend (%)  Contribution (%)  Min_Trend  Max_Trend  Min_Contribution  Max_Contribution
0  Hair/APDO           Asia            Tools         Balmain  Fragrance        2734489  3144662.4          50          15.000002         15.799760          0         15                 4                30
1  Hair/APDO  South America     Face Make-Up         Balmain  Fragrance        1739911  2000897.7          33          15.000003         10.198746          0         15                 4                30
2  Hair/APDO         Europe            Tools  Frederic Malle    Bronzer        3844769  3998559.8          59           4.000001         46.999958         -3          4                 4                30
3  Skin/Body  North 

## Maximize Margin

In [ ]:
import pandas as pd
import pulp

def maximize_margin(input_csv="Acme_Synthetic_Dataset.csv", output_csv="Optimized_Margin_Results.csv"):

    # 1. Load the dataset
    df = pd.read_csv(input_csv)

    # 2. Prompt the user for a desired total margin
    user_margin_target = float(input("Enter the desired total margin amount (in dollars): "))

    # 3. Initialize the LP problem to maximize margin
    prob = pulp.LpProblem("Maximize_Margin", pulp.LpMaximize)

    # 4. Create LP variables for new sales in each segment (no negative sales)
    sales_vars = pulp.LpVariable.dicts('Sales', df.index, lowBound=0)

    # 5. Objective function: sum of new_sales * (margin fraction)
    prob += pulp.lpSum([
        sales_vars[i] * (df.loc[i, 'Margin'] / 100.0)
        for i in df.index
    ]), "Total_Margin"

    # 6. Trend constraints for each segment
    for idx, row in df.iterrows():
        initial_sales = row['Initial_Sales']
        min_sales = initial_sales * (1 + row['Min_Trend'] / 100.0)
        max_sales = initial_sales * (1 + row['Max_Trend'] / 100.0)

        prob += sales_vars[idx] >= min_sales, f"Min_Trend_{idx}"
        prob += sales_vars[idx] <= max_sales, f"Max_Trend_{idx}"

    # 7. Contribution constraints within each Brand group
    grouped = df.groupby(['Portfolio', 'Geography', 'Category', 'Brand'])

    for name, group in grouped:
        indices = group.index
        total_sales_in_group = pulp.lpSum([sales_vars[i] for i in indices])

        for idx in indices:
            min_contrib_frac = df.loc[idx, 'Min_Contribution'] / 100.0
            max_contrib_frac = df.loc[idx, 'Max_Contribution'] / 100.0

            prob += sales_vars[idx] >= min_contrib_frac * total_sales_in_group, f"Min_Contribution_{idx}"
            prob += sales_vars[idx] <= max_contrib_frac * total_sales_in_group, f"Max_Contribution_{idx}"


    prob.solve()
    results = []
    for idx, row in df.iterrows():
        new_sales = sales_vars[idx].varValue

        # Calculate the applied trend
        initial_sales = row['Initial_Sales']
        applied_trend = ((new_sales / initial_sales) - 1) * 100 if initial_sales != 0 else 0

        # Find total brand sales for contribution calculation
        brand_group = grouped.get_group((row['Portfolio'], row['Geography'], row['Category'], row['Brand']))
        total_brand_sales = sum(sales_vars[i].varValue for i in brand_group.index)
        contribution_pct = (new_sales / total_brand_sales) * 100 if total_brand_sales != 0 else 0

        # Margin in dollars
        margin_dollars = new_sales * (row['Margin'] / 100.0)

        results.append({
            'Portfolio': row['Portfolio'],
            'Geography': row['Geography'],
            'Category': row['Category'],
            'Brand': row['Brand'],
            'Segment': row['Segment'],
            'Initial_Sales': initial_sales,
            'New_Sales': new_sales,
            'Margin (%)': row['Margin'],
            'Margin ($)': margin_dollars,
            'Applied_Trend (%)': applied_trend,
            'Contribution (%)': contribution_pct,
            'Min_Trend': row['Min_Trend'],
            'Max_Trend': row['Max_Trend'],
            'Min_Contribution': row['Min_Contribution'],
            'Max_Contribution': row['Max_Contribution']
        })

    results_df = pd.DataFrame(results)

    total_margin = results_df['Margin ($)'].sum()

    print(f"\nTotal Maximized Margin: ${total_margin:,.2f}")
    if total_margin >= user_margin_target:
        print(f"Great news! The optimized margin meets/exceeds your target by ${total_margin - user_margin_target:,.2f}.")
    else:
        print(f"Unfortunately, the optimized margin is below your target by ${user_margin_target - total_margin:,.2f}.")

    print("\nOptimized Results (first 10 rows):")
    print(results_df.head(10))

    results_df.to_csv(output_csv, index=False)
    print(f"\nDetailed results have been saved to '{output_csv}'.")



maximize_margin(input_csv="Acme_Synthetic_Dataset.csv",output_csv="Optimized_Margin_Results.csv")



    



Solver Status: Infeasible

Total Maximized Margin: $1,305,233,348.23
Great news! The optimized margin meets/exceeds your target by $1.23.

Optimized Results (first 10 rows):
                     Portfolio      Geography         Category           Brand    Segment  Initial_Sales   New_Sales  Margin (%)    Margin ($)  Applied_Trend (%)  Contribution (%)  Min_Trend  Max_Trend  Min_Contribution  Max_Contribution
0                    Hair/APDO           Asia            Tools         Balmain  Fragrance        2734489  3144662.40          50  1.572331e+06          15.000002         15.799760          0         15                 4                30
1                    Hair/APDO  South America     Face Make-Up         Balmain  Fragrance        1739911  2000897.70          33  6.602962e+05          15.000003         10.198746          0         15                 4                30
2                    Hair/APDO         Europe            Tools  Frederic Malle    Bronzer        3844769  39985

## Hit A Sales Target While Maximizing Margin

In [ ]:
import pandas as pd
import pulp

def hit_sales_target_maximize_margin(input_csv="Acme_Synthetic_Dataset.csv",output_csv="HitSalesTarget_MaxMargin_Results.csv"):
    df = pd.read_csv(input_csv)

    # Prompt user for a total sales target
    sales_target = float(input("Enter the desired total sales target (in dollars): "))


    # We want to maximize total margin (in dollars).
    prob = pulp.LpProblem("Hit_Sales_Target_Maximize_Margin", pulp.LpMaximize)

    # Create an LP variable for each segment's new sales
    sales_vars = pulp.LpVariable.dicts("NewSales", df.index, lowBound=0)

    # Objective: sum of new_sales_i * (margin_i / 100)
    prob += pulp.lpSum([
        sales_vars[i] * (df.loc[i, 'Margin'] / 100.0)
        for i in df.index
    ]), "Total_Margin"

    # sum of all new sales = user-specified target
    prob += pulp.lpSum(sales_vars[i] for i in df.index) == sales_target, "Sales_Target_Constraint"

    for idx, row in df.iterrows():
        initial_sales = row['Initial_Sales']
        min_sales = initial_sales * (1 + row['Min_Trend'] / 100.0)
        max_sales = initial_sales * (1 + row['Max_Trend'] / 100.0)

        prob += sales_vars[idx] >= min_sales, f"Min_Trend_{idx}"
        prob += sales_vars[idx] <= max_sales, f"Max_Trend_{idx}"

    # Group by (Portfolio, Geography, Category, Brand)
    grouped = df.groupby(['Portfolio', 'Geography', 'Category', 'Brand'])

    for name, group in grouped:
        group_indices = group.index
        total_sales_in_group = pulp.lpSum([sales_vars[i] for i in group_indices])

        for idx in group_indices:
            min_contrib_frac = df.loc[idx, 'Min_Contribution'] / 100.0
            max_contrib_frac = df.loc[idx, 'Max_Contribution'] / 100.0

            prob += sales_vars[idx] >= min_contrib_frac * total_sales_in_group, f"Min_Contribution_{idx}"
            prob += sales_vars[idx] <= max_contrib_frac * total_sales_in_group, f"Max_Contribution_{idx}"


    prob.solve()
    results = []
    for idx, row in df.iterrows():
        new_sales = sales_vars[idx].varValue

        # Applied trend
        initial_sales = row['Initial_Sales']
        if initial_sales != 0:
            applied_trend = (new_sales / initial_sales - 1) * 100.0
        else:
            applied_trend = 0.0

        # Contribution to brand total
        brand_group = grouped.get_group((
            row['Portfolio'],
            row['Geography'],
            row['Category'],
            row['Brand']
        ))
        total_brand_sales = sum(sales_vars[i].varValue for i in brand_group.index)
        contribution_pct = (new_sales / total_brand_sales) * 100 if total_brand_sales else 0.0

        # Margin in dollars
        margin_dollars = new_sales * (row['Margin'] / 100.0)

        results.append({
            'Portfolio': row['Portfolio'],
            'Geography': row['Geography'],
            'Category': row['Category'],
            'Brand': row['Brand'],
            'Segment': row['Segment'],
            'Initial_Sales': initial_sales,
            'New_Sales': new_sales,
            'Margin (%)': row['Margin'],
            'Margin ($)': margin_dollars,
            'Applied_Trend (%)': applied_trend,
            'Contribution (%)': contribution_pct,
            'Min_Trend': row['Min_Trend'],
            'Max_Trend': row['Max_Trend'],
            'Min_Contribution': row['Min_Contribution'],
            'Max_Contribution': row['Max_Contribution']
        })

    results_df = pd.DataFrame(results)

    # Calculate the total margin
    total_margin = results_df['Margin ($)'].sum()
    print(f"\nSales Target: ${sales_target:,.2f}")
    print(f"Total Achieved Margin: ${total_margin:,.2f}\n")

    # Display a sample of the results
    print("Optimized Results (first 10 rows):")
    print(results_df.head(10))

    # Save to CSV
    results_df.to_csv(output_csv, index=False)
    print(f"\nDetailed results saved to '{output_csv}'.")

hit_sales_target_maximize_margin(input_csv="Acme_Synthetic_Dataset.csv",output_csv="HitSalesTarget_MaxMargin_Results.csv")



Sales Target: $1,000,000,000,000.00
Total Achieved Margin: $300,430,471,774.13

Optimized Results (first 10 rows):
                     Portfolio      Geography         Category           Brand    Segment  Initial_Sales   New_Sales  Margin (%)    Margin ($)  Applied_Trend (%)  Contribution (%)  Min_Trend  Max_Trend  Min_Contribution  Max_Contribution
0                    Hair/APDO           Asia            Tools         Balmain  Fragrance        2734489  3144662.40          50  1.572331e+06          15.000002         14.413268          0         15                 4                30
1                    Hair/APDO  South America     Face Make-Up         Balmain  Fragrance        1739911  2000897.70          33  6.602962e+05          15.000003          8.924742          0         15                 4                30
2                    Hair/APDO         Europe            Tools  Frederic Malle    Bronzer        3844769  1932438.00          59  1.140138e+06         -49.738515         

## Hit A Margin Target While Maximizing Sales